In [127]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import time
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.models import Sequential
from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras_tqdm import TQDMNotebookCallback

from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


### Load data

In [128]:
    
# Getting the data:
dataset_name = 'DD'
width = 18
receptive_field = 10
PatchyConverter = GraphConverter(dataset_name, width, receptive_field)
mutag_tensor = PatchyConverter.graphs_to_Patchy_tensor()
# plt.imshow(mutag_tensor[0,:,:,2])

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
mutag_labels = dropbox_loader.get_graph_label()
mutag_labels = np.array(mutag_labels.graph_label)

# Assigning the data:
x_train, x_test, y_train, y_test = train_test_split(mutag_tensor, mutag_labels, test_size=0.10)
data = ((x_train,y_train),(x_test, y_test))
input_shape = x_train.shape[1:]
y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

DD tensor exists, loading it from Dropbox


In [ ]:
K.clear_session()


batch_size = 100
num_classes = 2
epochs = 200
# input image dimensionsa
input_shape = x_train.shape[1:]

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below


model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=input_shape))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(8, kernel_size=(5, 5), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(decay=1e-6),
              metrics=['accuracy'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])


Train on 1060 samples, validate on 118 samples
Epoch 1/200
1060/1060 [==============================] - 13s 12ms/step - loss: 0.6149 - acc: 0.6557 - val_loss: 0.5025 - val_acc: 0.8305
Epoch 2/200
1060/1060 [==============================] - 11s 11ms/step - loss: 0.5250 - acc: 0.7623 - val_loss: 0.4743 - val_acc: 0.8305
Epoch 3/200
1060/1060 [==============================] - 12s 12ms/step - loss: 0.4812 - acc: 0.7679 - val_loss: 0.4693 - val_acc: 0.8475
Epoch 4/200
1060/1060 [==============================] - 12s 12ms/step - loss: 0.4467 - acc: 0.8085 - val_loss: 0.4645 - val_acc: 0.8305
Epoch 5/200
1060/1060 [==============================] - 10s 10ms/step - loss: 0.3630 - acc: 0.8330 - val_loss: 0.4820 - val_acc: 0.8136
Epoch 6/200
1060/1060 [==============================] - 10s 10ms/step - loss: 0.2806 - acc: 0.9009 - val_loss: 0.5695 - val_acc: 0.8220
Epoch 7/200
1060/1060 [==============================] - 10s 10ms/step - loss: 0.2025 - acc: 0.9443 - val_loss: 0.5418 - val_acc: 0

1060/1060 [==============================] - 11s 11ms/step - loss: 2.1977e-04 - acc: 1.0000 - val_loss: 1.4069 - val_acc: 0.7712
Epoch 60/200
1060/1060 [==============================] - 13s 12ms/step - loss: 1.5216e-04 - acc: 1.0000 - val_loss: 1.3973 - val_acc: 0.7712
Epoch 61/200
1060/1060 [==============================] - 12s 11ms/step - loss: 1.4880e-04 - acc: 1.0000 - val_loss: 1.3983 - val_acc: 0.7712
Epoch 62/200
1060/1060 [==============================] - 11s 11ms/step - loss: 1.9055e-04 - acc: 1.0000 - val_loss: 1.4047 - val_acc: 0.7712
Epoch 63/200
1060/1060 [==============================] - 11s 10ms/step - loss: 1.5437e-04 - acc: 1.0000 - val_loss: 1.3636 - val_acc: 0.7627
Epoch 64/200
1060/1060 [==============================] - 11s 10ms/step - loss: 1.4006e-04 - acc: 1.0000 - val_loss: 1.3785 - val_acc: 0.7627
Epoch 65/200
1060/1060 [==============================] - 12s 11ms/step - loss: 2.0301e-04 - acc: 1.0000 - val_loss: 1.4094 - val_acc: 0.7627
Epoch 66/200
1060/1

Epoch 117/200
1060/1060 [==============================] - 11s 11ms/step - loss: 7.8635e-05 - acc: 1.0000 - val_loss: 1.7141 - val_acc: 0.7966
Epoch 118/200
1060/1060 [==============================] - 11s 11ms/step - loss: 7.8337e-05 - acc: 1.0000 - val_loss: 1.7414 - val_acc: 0.7966
Epoch 119/200
1060/1060 [==============================] - 11s 11ms/step - loss: 4.2556e-05 - acc: 1.0000 - val_loss: 1.6582 - val_acc: 0.7712
Epoch 120/200
1060/1060 [==============================] - 12s 11ms/step - loss: 1.0410e-04 - acc: 1.0000 - val_loss: 1.6317 - val_acc: 0.7458
Epoch 121/200
1060/1060 [==============================] - 11s 11ms/step - loss: 5.0189e-05 - acc: 1.0000 - val_loss: 1.6507 - val_acc: 0.7712
Epoch 122/200
1060/1060 [==============================] - 11s 10ms/step - loss: 2.4589e-05 - acc: 1.0000 - val_loss: 1.6987 - val_acc: 0.7966
Epoch 123/200
1060/1060 [==============================] - 12s 11ms/step - loss: 3.8245e-05 - acc: 1.0000 - val_loss: 1.7017 - val_acc: 0.7966

In [ ]:
model.summary()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(epochs), history.acc)
plt.legend('train data',loc='lower center')
plt.plot(range(epochs), history.val_acc)
plt.legend('test data')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [92]:
# Get the training Arguments
args = CapsuleTrainingParameters()

# Get the NN Parameters 
capsule_params= CapsuleParameters()
# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'
capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
# Parameters of the capsule:
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 512
decoder_params['second_dense'] = 1024
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)



In [93]:
#args.data_augmentation = True
patchy_classifier = GraphClassifier(input_shape)
patchy_classifier.build_the_graph(capsule_params)

args.epochs = 200
args.verbose = 1
patchy_classifier.train(data,args)
model = patchy_classifier.train_model

time to generate the model: 0.2900218963623047
Train on 1060 samples, validate on 118 samples
Epoch 1/100
1060/1060 [==============================] - 7s 6ms/step - loss: 0.5178 - capsnet_loss: 0.4204 - decoder_loss: 0.2485 - capsnet_acc: 0.5868 - val_loss: 0.4132 - val_capsnet_loss: 0.3174 - val_decoder_loss: 0.2445 - val_capsnet_acc: 0.5085

Epoch 00001: val_capsnet_acc improved from -inf to 0.50847, saving model to ./result/weights-01.h5
Epoch 2/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.3305 - capsnet_loss: 0.2408 - decoder_loss: 0.2288 - capsnet_acc: 0.5953 - val_loss: 0.3112 - val_capsnet_loss: 0.2354 - val_decoder_loss: 0.1934 - val_capsnet_acc: 0.5085

Epoch 00002: val_capsnet_acc did not improve from 0.50847
Epoch 3/100
1060/1060 [==============================] - 7s 6ms/step - loss: 0.2537 - capsnet_loss: 0.2107 - decoder_loss: 0.1098 - capsnet_acc: 0.5953 - val_loss: 0.2282 - val_capsnet_loss: 0.2171 - val_decoder_loss: 0.0283 - val_capsnet_acc: 0

1060/1060 [==============================] - 6s 6ms/step - loss: 0.0085 - capsnet_loss: 0.0038 - decoder_loss: 0.0120 - capsnet_acc: 1.0000 - val_loss: 0.2609 - val_capsnet_loss: 0.2558 - val_decoder_loss: 0.0130 - val_capsnet_acc: 0.6102

Epoch 00026: val_capsnet_acc did not improve from 0.64407
Epoch 27/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0081 - capsnet_loss: 0.0034 - decoder_loss: 0.0120 - capsnet_acc: 1.0000 - val_loss: 0.2567 - val_capsnet_loss: 0.2516 - val_decoder_loss: 0.0131 - val_capsnet_acc: 0.6356

Epoch 00027: val_capsnet_acc did not improve from 0.64407
Epoch 28/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0078 - capsnet_loss: 0.0031 - decoder_loss: 0.0120 - capsnet_acc: 1.0000 - val_loss: 0.2588 - val_capsnet_loss: 0.2537 - val_decoder_loss: 0.0130 - val_capsnet_acc: 0.6102

Epoch 00028: val_capsnet_acc did not improve from 0.64407
Epoch 29/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.

1060/1060 [==============================] - 6s 6ms/step - loss: 0.0062 - capsnet_loss: 0.0016 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00052: val_capsnet_acc did not improve from 0.64407
Epoch 53/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0062 - capsnet_loss: 0.0016 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00053: val_capsnet_acc did not improve from 0.64407
Epoch 54/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.0062 - capsnet_loss: 0.0015 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00054: val_capsnet_acc did not improve from 0.64407
Epoch 55/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.

1060/1060 [==============================] - 6s 6ms/step - loss: 0.0061 - capsnet_loss: 0.0015 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00078: val_capsnet_acc did not improve from 0.64407
Epoch 79/100
1060/1060 [==============================] - 7s 6ms/step - loss: 0.0061 - capsnet_loss: 0.0015 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00079: val_capsnet_acc did not improve from 0.64407
Epoch 80/100
1060/1060 [==============================] - 7s 6ms/step - loss: 0.0061 - capsnet_loss: 0.0015 - decoder_loss: 0.0119 - capsnet_acc: 1.0000 - val_loss: 0.2584 - val_capsnet_loss: 0.2532 - val_decoder_loss: 0.0132 - val_capsnet_acc: 0.6356

Epoch 00080: val_capsnet_acc did not improve from 0.64407
Epoch 81/100
1060/1060 [==============================] - 6s 6ms/step - loss: 0.

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [103]:
pd.DataFrame(pd.Series(('weg',12414),index=[4,5])).transpose()

,4,5
0,weg,12414


In [105]:
patchy_classifier.log_file

'./result//log_test.csv'

In [106]:
args.log_filename

'/log_test.csv'

In [111]:
path =os.path.join(args.save_dir,args.log_filename[1:])

In [120]:
x =pd.read_csv(path).loc[:,['epoch', 'capsnet_acc', 'val_capsnet_acc']].iloc[-1,:]

In [94]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 18, 10, 82)   0                                            
__________________________________________________________________________________________________
relu (Conv2D)                   (None, 10, 2, 256)   1700608     input_1[0][0]                    
__________________________________________________________________________________________________
primarycap_conv2d (Conv2D)      (None, 5, 1, 256)    262400      relu[0][0]                       
__________________________________________________________________________________________________
primarycap_reshape (Reshape)    (None, 160, 8)       0           primarycap_conv2d[0][0]          
__________________________________________________________________________________________________
primarycap

In [ ]:
file_path = '/Users/marcelogutierrez/Projects/Gamma/capsuleSans/PatchyCapsules/result/log.csv'
df = pd.read_csv(file_path,index_col=0)

In [ ]:
pd.concat([test_acc, test_acc2],1)